<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5A_S3_BERT%E7%94%A8%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT データセットの作成 S3 </center>

###<center>リツイート・@mention・URLの除去</center>


#0. 作業環境の準備

##0.1. google driveのマウント

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##0.2. カレントディレクトリへ移動

In [21]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在時刻の取得

In [22]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  DATETIME_now = DATETIME_now.strftime("%Y-%m-%d")
  #DATETIME_now = DATETIME_now.strftime("%Y/%m/%d %H:%M:%S")
  #print(DATETIME_now)

  return DATETIME_now

print(pull_Datetime())

2021-10-14


##option. データセットの指定

In [23]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v5_forBERT.csv'

#1. データセットのロード

In [24]:
import pandas as pd
df_v0=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df_v0.shape}')

df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

データサイズ： (483083, 16)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,RT @det_kirigakure: #毎日龍驤\n謹賀新年、明けましておめでとうございま...,2017-12-31 15:23:39+00:00,INTJ,I,N,T,J
1,RT @nunnu_zero: 今流行りの壺をうちの子にプレゼントしてみた https://...,2017-12-31 10:42:48+00:00,INTJ,I,N,T,J
2,RT @wahiko94: 下着姿の加賀殿を描いたでござる https://t.co/rX2...,2017-12-31 10:41:44+00:00,INTJ,I,N,T,J
3,RT @whimgreen: ユニコーンちゃんに膝枕してもらいたい人生だった #アズールレー...,2017-12-30 14:01:55+00:00,INTJ,I,N,T,J
4,RT @whimgreen: スカートの中見せてくれるのかと期待して「すけべ♪」って言われた...,2017-12-30 14:01:38+00:00,INTJ,I,N,T,J


#2. データの加工

##2.1. 必要なデータ（文章とラベル）の抽出

In [25]:
df_v0.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df_v0.dropna(how='any', axis=0, inplace=True)

df=df_v0
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,RT @det_kirigakure: #毎日龍驤\n謹賀新年、明けましておめでとうございま...,2017-12-31 15:23:39+00:00,INTJ,I,N,T,J
1,RT @nunnu_zero: 今流行りの壺をうちの子にプレゼントしてみた https://...,2017-12-31 10:42:48+00:00,INTJ,I,N,T,J
2,RT @wahiko94: 下着姿の加賀殿を描いたでござる https://t.co/rX2...,2017-12-31 10:41:44+00:00,INTJ,I,N,T,J
3,RT @whimgreen: ユニコーンちゃんに膝枕してもらいたい人生だった #アズールレー...,2017-12-30 14:01:55+00:00,INTJ,I,N,T,J
4,RT @whimgreen: スカートの中見せてくれるのかと期待して「すけべ♪」って言われた...,2017-12-30 14:01:38+00:00,INTJ,I,N,T,J


##2.2. データからリツイートの投稿を除去

In [26]:
#dfXXX=df['text'].str.extract(r'RT\ @(.*)', expand=True) 
df = df[~df['text'].str.contains(r'RT\ @(.*)')]
#dfXXX = df[~df['text'].str.startswith('RT ')]
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
5,@ha_ma あけおめなのよ。ことよろなのよ。,2017-12-30 12:02:23+00:00,INTJ,I,N,T,J
11,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
100,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
104,Vステなら大丈夫だったのになぁ…\nまぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
105,すげー！！耐えた！！ #鉄腕DASH,2017-11-19 10:32:29+00:00,INTJ,I,N,T,J


In [27]:
# import random
#  #データの抽出
# df=df.sample(n=10000)

##2.3. @mention・URLの削除

In [37]:
import re
import numpy as np

def remove_MENTIONandURL(text):
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  element_mention=None
  element_URL=None
  for idx, element in enumerate(list_text):
    if re.match(r'(^)(@)[^a-zA-Z0-9_]*', element):
      element_mention=element
    if element.startswith('https:'):
      element_URL=element
      
  #print(element_mention, element_URL)
  if element_mention==None:
    pass
  else:
    list_text.remove(element_mention)
  if element_URL==None:
    pass
  else:
    list_text.remove(element_URL)

  Z=' '.join(list_text)
  
  return Z

df['text'] = df['text'].map(remove_MENTIONandURL)
df.dropna(how='any', axis=0, inplace=True)
df['text']

270009    「今年は納税額が少なそうですね」と税理士さんからメールが来た。  産休で8月から休んでいるの...
474307                         簿記の勉強しないとまじでやばいけどツイッター閉じられない
263903                                       孤独のグルメが生放送ドラマに
58066          デオンくんちゃん、マリー 並行で婦長のレベルもマックスにできた（これは絆が上がったママ）
429909                                       今日も生き延びることができた
                                ...                        
186605    つか、最初に発見した村がツンドラ地帯のすぐ近くで、畑が機能してないという…お前ら何食って生き...
448763                                          栃木戻ってまいりました
34412           本日の成果：日本国憲法序文を3分半くらいでガシガシと一気にタイピングできるようになった
56261                    はー、やっぱこの都会オーラ？ってやつ？隠せないよね～、出ちゃうよね～
176898                                     お疲れ…狙いはさえはん？なお…？
Name: text, Length: 10000, dtype: object

#3. データの保存

##3.1. 手法4までで作成した投稿を更新

In [29]:
df_all=df
print(f'データサイズ： {df_all.shape}')

FileName=pull_Datetime()
df.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6S3_all_%s.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v6S3_all_%s.csv' % FileName)

データサイズ： (367445, 11)
./Assets/Assets_Output/Tweet_Datasets/v6S3_all_2021-10-14.csv


##3.2. データセットS3の作成

In [30]:
import random
 #データの抽出
df=df.sample(n=10000)

In [31]:
FileName=pull_Datetime()
df.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6S3_forBERT_%s.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v6S3_forBERT_%s.csv' % FileName)

./Assets/Assets_Output/Tweet_Datasets/v6S3_forBERT_2021-10-14.csv


##3.3. 記録用に各指標のユーザー数を表示

In [32]:
def num_Type_Index(element):
  num_Users_Type=df.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()

  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

label_EorI
E    1544
I    8456
dtype: int64

label_NorS
N    5702
S    4298
dtype: int64

label_ForT
F    6652
T    3348
dtype: int64

label_JorP
J    3418
P    6582
dtype: int64



#Option 最大単語数の計算

In [33]:
!pip install transformers

!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 2.9 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 31.0 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 33.4 MB/s 
     |████████████████████████████████| 3.3 MB 26.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 490 kB 6.8 MB/s 
     |████████████████████████████████| 13.4 MB 195 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=0f2603e5c2fde20c400754d0ed98e1479688c5b076a1b7723eaea54633c47c6f
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [34]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [35]:
from tqdm.notebook import tqdm
import numpy as np

def num_max_words(TYPE_INDEX):
  sentences= df['text'].values #sentences = df.sentence.values
  labels= df['%s' % TYPE_INDEX].values #labels = df.label.values

  # 最大単語数の確認
  max_len = []
  # 1文づつ処理
  len_sentences = len(sentences)
  for sent in tqdm(sentences):
      # Tokenizeで分割
      token_words = tokenizer.tokenize(sent)
      # 文章数を取得してリストへ格納
      max_len.append(len(token_words))
  # 最大の値を確認
  len_max=max(max_len)
  #len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
  #len_q75=int(len_q75)
  print('最大単語数: ', max(max_len))

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_max_words(element)

  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  106


  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  106


  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  106


  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  106
